In [1]:
from Bio.KEGG import REST
import pandas as pd

In [3]:
def get_kos(pathway):
    kos = REST.kegg_link('ko', pathway).read()
    kos = list(kos.split('\n'))
    kos = [ko.split('\t')[1] for ko in kos[:-1]]
    return kos[:10]

In [11]:
def get_genes(ko):
    genes = REST.kegg_link('genes', ko).read()
    genes = list(genes.split('\n'))
    genes = [gene.split('\t')[1] for gene in genes[:-1] if is_prokaryotic(gene)]
    return genes[:10]

In [4]:
def is_prokaryotic(gene):
    # if gene[:2] in prokaryotes_codes:
    if gene[:2] == 'dme':
        return True 

In [2]:
pathways = REST.kegg_list('pathway').read()
pathways = list(pathways.split('\n'))
pathways = [pathway.split('\t')[0] for pathway in pathways]

In [6]:
organisms = REST.kegg_list('organism').read()
organisms = list(organisms.split('\n'))
prokaryotes_codes = [organism.split('\t')[1] for organism in organisms if 'Prokaryotes' in organism]

In [10]:
genes[:10]

[]

In [9]:
prokaryotes_codes[:3]

['eco', 'ecj', 'ecd']

In [12]:
data = {}
for pathway in pathways[:2]:
    print(pathway)
    kos = get_kos(pathway)
    for ko in kos:
        genes = get_genes(ko)
        print(genes)
    data.update({ko: genes})

path:map00010
['dme:Dmel_CG3481', 'der:6540581', 'dse:6611292', 'dsi:Dsimw501_GD23968', 'dya:Dyak_GE19037', 'dya:Dyak_GE10683', 'dan:6492907', 'dsr:110186092', 'dpo:Dpse_GA17214', 'dpe:6593284']
['hsa:10327', 'ptr:741418', 'pps:100990701', 'ggo:101124639', 'pon:100173796', 'nle:100581229', 'mcc:693380', 'mcf:102143796', 'csab:103224909', 'rro:104662135']
['hsa:160287', 'hsa:92483', 'hsa:3939', 'hsa:3945', 'hsa:3948', 'ptr:466460', 'ptr:451063', 'ptr:747141', 'ptr:451064', 'ptr:465337']
['dtx:ATSB10_32210', 'dko:I596_887', 'rbd:ALSL_0372', 'gho:AL542_17210', 'gho:AL542_11380', 'pae:PA1982', 'paev:N297_2045', 'paei:N296_2045', 'pau:PA14_38860', 'pap:PSPA7_3312']
['hsa:128', 'ptr:743928', 'pps:100969269', 'ggo:101133633', 'pon:100450815', 'nle:100592499', 'mcc:707258', 'mcf:102122307', 'csab:103236693', 'rro:104664094']
['hsa:217', 'hsa:224', 'hsa:219', 'ptr:464931', 'ptr:454505', 'ptr:107967644', 'pps:100969562', 'pps:100970208', 'pps:100981893', 'ggo:101130617']
['hsa:221', 'hsa:222', '

In [ ]:
data

{'ko:K00138': [], 'ko:K00164': []}

In [ ]:
df = pd.DataFrame(data)
# columns = [('path:map00010','ko:K00001'), ('path:map00020','ko:K00002')]
# df.columns = pd.MultiIndex.from_tuples(columns)

In [ ]:
df

,path:map00010,path:map00020
ko:K00121,"[hsa:128, ptr:743928, pps:100969269]",NaN
ko:K00031,NaN,"[hsa:3417, hsa:3418, ptr:460113]"
